# PolarsとPython間のデータ変換

In [2]:
import polars as pl
from helper.jupyter import row

## PythonからPolarsに

### リストと辞書の組み合わせ

次のプログラムは、異なるPythonのデータ構造（辞書のリスト、リストの辞書、リストのリスト）を使用して`DataFrame`を作成します。

1. 辞書のリスト(`list[dict]`): 辞書のキーが列名となり、リスト内の各辞書が1行分のデータとなります。
2. リストの辞書(`dict[list]`): 辞書のキーが列名となり、各リストの要素がその列に対応するデータになります。
3. リストのリスト(`list[list]`): `schema`引数で列名を指定します。
   * `orient`引数は`'row'`の場合は、データの方向が行単位であることを指定して、内部の一つリストが1行分のデータとなります。
   * `orient`引数は`'col'`の場合は、データの方向が列単位であることを指定して、内部の一つリストが1列分のデータとなります。この例では内部リストのデータ型一致しないので、`strict=False`で自動型変換を有効にします。

In [15]:
# list[dict]
dict_in_list = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
    {"name": "Charlie", "age": 35}
]
df1 = pl.DataFrame(dict_in_list)

# dict[list]
list_in_dict = {
    "name": ["Alice", "Bob", "Charlie"],
    "age": [30, 25, 35],
}
df2 = pl.DataFrame(list_in_dict)

# list[list]
list_in_list = [
    ["Alice", 30],
    ["Bob", 25],
    ["Charlie", 35]
]
columns = ["name", "age"]  # カラム名を指定
df3 = pl.DataFrame(list_in_list, schema=columns, orient='row')
df4 = pl.DataFrame(list_in_list, schema=['p1', 'p2', 'p3'], orient='col', strict=False)
row(df1, df2, df3, df4)

name,age
str,i64
"""Alice""",30
"""Bob""",25
"""Charlie""",35
name,age
str,i64
"""Alice""",30
"""Bob""",25
"""Charlie""",35
name,age


次の`data` は辞書形式で、次のようなデータを持っています：

 - `"point"` キーの値は辞書のリスト (`list[dict]`) で、各辞書には`x` と `y` の2つのキーが含まれています。
 - `"weight"` キーの値は整数のリスト (`list[int]`) です。

データフレームに変換するとき、外側の辞書のキーは列名になり、`point`列の要素は`Struct`型（構造体）に変換されます。

In [65]:
data = {
    "point": [{"x": 1, "y": 2}, {"x": 3, "y": 4}, {"x": 5, "y": 6}],
    "weight": [5, 4, 8],
}
pl.DataFrame(data)

point,weight
struct[2],i64
"{1,2}",5
"{3,4}",4
"{5,6}",8


### NumPyの配列

NumPyの配列を扱う際、以下のようにlistとNumPy配列と互換性を持ちます。

* `dict[list]`と`dict[1次元配列]`は同じ扱い
* `list[list]`と２次元配列は同じ扱い

In [25]:
import numpy as np
array_in_dict = {
    "x": np.array([1, 3, 5]),
    "y": np.array([2, 4, 6]),
}

df1 = pl.DataFrame(array_in_dict)

array_2d = np.array([[1, 2], [3, 4], [5, 6]])
df2 = pl.DataFrame(array_2d, schema=['x', 'y'], orient='row')
df3 = pl.DataFrame(array_2d, schema=['p1', 'p2', 'p3'], orient='col')
row(df1, df2, df3)

shape: (3, 2) x y i32 i32 1 2 3 4 5 6 
 shape: (3, 2) x y i32 i32 1 2 3 4 5 6 
 shape: (2, 3) p1 p2 p3 i32 i32 i32 1 3 5 2 4 6

1次元の構造化配列をデータフレームに変換する場合は、配列の各フィールドはデータフレームの各列になります。

In [66]:
arr = np.array([
    (1, 30),
    (2, 25),
    (3, 35)], dtype=[('x', 'i2'), ('y', 'i2')])

pl.DataFrame(arr)

x,y
i16,i16
1,30
2,25
3,35


### Seriesを含むデータ

`pl.Series` を扱う場合、`list[Series]` や `dict[Series]` の形式をデータフレームに変換することがよくあります。どちらの場合も、それぞれの `Series` はデータフレームの列になりますが、列名の扱いが異なります。

- `list[Series]`: 列名は `Series` の名前がそのまま使われます。
- `dict[Series]`: 列名は辞書のキーが使われます。

In [67]:
sx = pl.Series('x', [1, 2, 3])
sy = pl.Series('y', [4, 5, 6])

df1 = pl.DataFrame([sx, sy])
df2 = pl.DataFrame({'A':sx, 'B':sy})
row(df1, df2)

x,y
i64,i64
1,4
2,5
3,6
A,B
i64,i64
1,4
2,5
3,6


### pl.from_*()関数

`from_` で始まる関数は、さまざまなデータ型をデータフレームに変換するために使用されます。これらの関数を利用すると、意図しないデータ変換が発生しにくく、コードのロバスト性を向上させることができます。

- `pl.from_dict()`: `dict[list]` のデータから変換
- `pl.from_dicts()`: `list[dict]` のデータから変換
- `pl.from_numpy()`: NumPy の配列から変換
- `pl.from_records()`: `list[list]` のデータから変換

## PolarsからPythonに

本節は、データフレームから列、行、或いは単一の値を取得する方法について説明します。

In [3]:
df = pl.DataFrame(
    {
        "a": [3, 3, 3, 4],
        "b": [4, 12, 6, 7],
        "g": ['A', 'B', 'A', 'B']
    }
)

### 列を取得

PolarsでDataFrameから列データをSeriesとして取得する方法はいくつかあります。

* `DataFrame.to_series()`: インデックスで列を取得します。
* `DataFrame.get_column()`: 列名で列を取得します。
* `DataFrame.get_columns()`: すべての列を取得します。
* `DataFrame.iter_columns()`: 列のイテレーターを取得します。

`DataFrame.to_series()` メソッドを使用すると、指定したインデックスに基づいて列を Series として取得できます。`DataFrame.get_column()` メソッドを使用すると、列名を指定して Series を取得できます。`DataFrame["column_name"]`のような辞書形式で列名を指定して Seriesを取得することもできます。

In [5]:
s1 = df.to_series(0)
s2 = df.get_column('b')
s3 = df['g']
row(s1, s2, s3)

a
i64
3
3
3
4
b
i64
4
12
6


`DataFrame.get_columns()` メソッドは、DataFrame 内のすべての列を Series のリストとして取得します。

In [6]:
sa, sb, sg = df.get_columns()
row(sa, sb, sg)

a
i64
3
3
3
4
b
i64
4
12
6


`DataFrame.iter_columns()`は、DataFrame内のすべての列を一つずつ返します。

In [8]:
for col in df.iter_columns():
    print(col.name, col.to_list())

a [3, 3, 3, 4]
b [4, 12, 6, 7]
g ['A', 'B', 'A', 'B']


### to_*()メソッド

`Series.to_numpy()`または`Series.to_list()`メソッドを使用すると、`Series`オブジェクトをNumPyの配列やリストに変換することができます。

In [16]:
print(f'{sa.to_numpy() = }')
print(f'{sa.to_list() = }')

sa.to_numpy() = array([3, 3, 3, 4], dtype=int64)
sa.to_list() = [3, 3, 3, 4]


`DataFrame.to_numpy()`でNumPyの配列に変換することができます。デフォルトはすべての値を一番上位のデータ型に変換します。数値と文字列混在のデータの場合は、`dtype=object`の配列になります。

In [15]:
df.to_numpy()

array([[3, 4, 'A'],
       [3, 12, 'B'],
       [3, 6, 'A'],
       [4, 7, 'B']], dtype=object)

`structured`引数を`True`にすれば、構造化配列に変換します。

In [14]:
df.to_numpy(structured=True)

array([(3,  4, 'A'), (3, 12, 'B'), (3,  6, 'A'), (4,  7, 'B')],
      dtype=[('a', '<i8'), ('b', '<i8'), ('g', '<U1')])

`to_dict()`メソッドで`dict[Series]`型の辞書に変換します。

In [29]:
df.to_dict()

{'a': shape: (4,)
 Series: 'a' [i64]
 [
 	3
 	3
 	3
 	4
 ],
 'b': shape: (4,)
 Series: 'b' [i64]
 [
 	4
 	12
 	6
 	7
 ],
 'g': shape: (4,)
 Series: 'g' [str]
 [
 	"A"
 	"B"
 	"A"
 	"B"
 ]}

`to_dicts()`メソッドで`list[dict]`型のリストに変換します。

In [30]:
df.to_dicts()

[{'a': 3, 'b': 4, 'g': 'A'},
 {'a': 3, 'b': 12, 'g': 'B'},
 {'a': 3, 'b': 6, 'g': 'A'},
 {'a': 4, 'b': 7, 'g': 'B'}]

### 行を取得

- **`DataFrame.row(index)`**: 特定のインデックスにある行をタプルとして取得します。
- **`DataFrame.rows()`**: DataFrame のすべての行をタプルのリストとして取得します。
- **`DataFrame.iter_rows()`**: DataFrame の各行に対してイテレーションを行い、各行をタプルとして返します。
- **`DataFrame.rows_by_key()`**: 指定したキーに基づいて行をグループ化し、各グループに属する行をタプルのリストとして返します。

`DataFrame.row(index)` は、指定したインデックス（行番号）に対応する行を取得するためのメソッドです。このメソッドは、特定の行をタプル形式で返します。

In [17]:
df.row(2)

(3, 6, 'A')

`named`引数を`True`にすることで、列名を含む辞書形式で返します。

In [18]:
df.row(2, named=True)

{'a': 3, 'b': 6, 'g': 'A'}

`by_predicate`で演算式がTrueになる行を取得することができます。複数の行は条件に満たす場合は、エラーになります。

In [19]:
df.row(by_predicate=pl.col.a == pl.col.a.max())

(4, 7, 'B')

`DataFrame.rows()` は、DataFrame 内のすべての行をタプルのリストとして取得するメソッドです。DataFrame 全体のデータを行単位で操作したい場合に便利です。又、`DataFrame.iter_rows()` は各行に対してイテレーションを行うためのメソッドです。

In [20]:
df.rows()

[(3, 4, 'A'), (3, 12, 'B'), (3, 6, 'A'), (4, 7, 'B')]

In [21]:
df.rows(named=True)

[{'a': 3, 'b': 4, 'g': 'A'},
 {'a': 3, 'b': 12, 'g': 'B'},
 {'a': 3, 'b': 6, 'g': 'A'},
 {'a': 4, 'b': 7, 'g': 'B'}]

In [22]:
for row in df.iter_rows():
    print(row)

(3, 4, 'A')
(3, 12, 'B')
(3, 6, 'A')
(4, 7, 'B')


`DataFrame.rows_by_key()` は、指定したキー（列）に基づいて DataFrame 内の行をグループ化し、そのキーに対応する行をタプルのリストとして返すメソッドです。

In [23]:
df.rows_by_key('g')

defaultdict(list, {'A': [(3, 4), (3, 6)], 'B': [(3, 12), (4, 7)]})

In [24]:
df.rows_by_key('g', named=True)

defaultdict(list,
            {'A': [{'a': 3, 'b': 4}, {'a': 3, 'b': 6}],
             'B': [{'a': 3, 'b': 12}, {'a': 4, 'b': 7}]})

### 単一の値

`DataFrame.item()`は、DataFrameから単一の値を取得するためのメソッドです。

In [25]:
df.item(2, 'a')

3

DataFrameに値が一つだけ含まれる場合、引数を省略できます。列の統計値を取得して、それを別の計算に利用する際によく使用されます。

In [33]:
df.select(pl.col('a').mean()).item() / 100

0.0325